1. Таблица с данными о пользователях (user):

id - уникальный идентификатор пользователя (primary key)

gender - пол

age - возраст

country - страна

city - город

exp_group - экспериментальная группа

os - операционная система

source - источник трафика

2. Таблица с данными о постах (post):

id - уникальный идентификатор поста (primary key)

text - текст поста

topic - тема поста

3. Таблица с данными о действиях пользователей (feed_action):

user_id (——>) user (id)- идентификатор пользователя     

post_id (——>) post (id)- идентификатор поста     

action - совершенное в сети действие     

time- время действия

In [2]:
import pandas as pd
from sqlalchemy import create_engine

connect = create_engine("") # подключение к базе

In [3]:
user = pd.read_sql(
    """SELECT * FROM "user" """,
    con=connect
)

user.head()

,id,gender,age,country,city,exp_group,os,source
0,200,1,34,Russia,Degtyarsk,3,Android,ads
1,201,0,37,Russia,Abakan,0,Android,ads
2,202,1,17,Russia,Smolensk,4,Android,ads
3,203,0,18,Russia,Moscow,1,iOS,ads
4,204,0,36,Russia,Anzhero-Sudzhensk,3,Android,ads


In [4]:
topic = pd.read_sql(
    """SELECT DISTINCT topic FROM "post" """,
    con=connect
)

topic.head()

,topic
0,tech
1,covid
2,movie
3,entertainment
4,politics


In [5]:
pd.read_sql(
    """SELECT "id", "city" 
       FROM "user" 
       WHERE "age" > 30 AND "os" = 'iOS' """,
    con=connect
).head(5)

,id,city
0,212,Podolsk
1,222,Ulyanovsk
2,224,Yekaterinburg
3,226,Moscow
4,243,Ekibastuz


In [6]:
pd.read_sql(
    """SELECT "country", ROUND(AVG("age"), 2)
       FROM "user" 
       GROUP BY "country"  """,
    con=connect
).head(5)

,country,round
0,Azerbaijan,26.78
1,Belarus,27.28
2,Cyprus,28.07
3,Estonia,28.15
4,Finland,27.13


In [7]:
pd.read_sql(
    """SELECT id
       FROM "user" 
       WHERE "country" != 'Russia' AND ("exp_group" not in (0,3)  OR "city" = 'Minsk') """,
    con=connect
).head(5)

,id
0,211
1,213
2,216
3,218
4,228


In [8]:
pd.read_sql(
    """SELECT exp_group, os, count(id) AS total_users, MAX(age) AS max_age, MIN(age) AS min_age
       FROM "user" 
       GROUP BY exp_group, os  """,
    con=connect
).to_csv('out.csv', index=False)

In [9]:
pd.read_sql(
    """SELECT "topic", MAX(LENGTH("text"))
       FROM "post" 
       GROUP BY "topic"
       ORDER BY 2
       """,
    con=connect
)

,topic,max
0,covid,148
1,business,5378
2,sport,9438
3,movie,10234
4,tech,16063
5,entertainment,19004
6,politics,25392


In [10]:
pd.read_sql(
    """SELECT "country", count(id)
       FROM "user" 
       GROUP BY "country"
       HAVING count(id) > 1000
       ORDER BY 2
       """,
    con=connect
)

,country,count
0,Azerbaijan,1542
1,Finland,1599
2,Turkey,1606
3,Kazakhstan,3172
4,Belarus,3293
5,Ukraine,8273
6,Russia,143035


In [11]:
pd.read_sql(
    """SELECT "exp_group", count(id)
       FROM "user" 
       WHERE "city" = 'Moscow'
       GROUP BY "exp_group"
       HAVING avg(age) > 27.2
       ORDER BY 2
       """,
    con=connect
)

,exp_group,count
0,3,4378
1,1,4414


In [12]:
pd.read_sql(
    """SELECT "topic", count("text")
       FROM "post" 
       GROUP BY "topic"
       ORDER BY 2
       """,
    con=connect
)

,topic,count
0,entertainment,386
1,tech,401
2,politics,417
3,sport,510
4,business,510
5,covid,1799
6,movie,3000


In [13]:
pd.read_sql(
    """SELECT "id", "exp_group", "age"
       FROM "user" 
       WHERE city = 'Voronezh'
       ORDER BY 3 DESC, 2
       """,
    con=connect
).head(2)

,id,exp_group,age
0,89990,0,71
1,2441,3,71


In [17]:
pd.read_sql(
    """SELECT f.post_id, f.time, u.age, u.os
         FROM "user" u
        INNER JOIN "feed_action" f
                ON u.id = f.user_id
               AND f.action = 'like'
        
       WHERE u.city = 'Omsk'
       ORDER BY 2 DESC
       LIMIT 100
       """,
    con=connect
).to_csv('out_13.csv', index=False)

In [23]:
pd.read_sql(
    """SELECT u.city, count(f.action)
         FROM "user" u
        INNER JOIN "feed_action" f
                ON u.id = f.user_id
               AND f.action = 'view'
               AND DATE(f.time) = '2021-12-01'
        INNER JOIN "post" p
                ON p.id = f.post_id
               AND p.topic = 'covid'
       WHERE u.age = '36'
       GROUP BY u.city
       ORDER BY 2
       """,
    con=connect
).head(1)

,city,count
0,Arkhangelsk,5


In [25]:
pd.read_sql(
    """SELECT p.id, count(f.action), count(DISTINCT f.user_id), MAX(f.time)
         FROM "post" p
        INNER JOIN "feed_action" f
                ON p.id = f.post_id
               AND f.action = 'like'
       GROUP BY p.id
       ORDER BY 2 DESC
       """,
    con=connect
).head(3)

,id,count,count,max
0,4866,31714,31714,2022-01-01 13:57:43
1,5127,29637,29637,2022-01-05 13:52:17
2,6448,27939,27939,2022-01-22 05:50:39
